## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-13-10-55-20 +0000,bbc,Kelly sues Hegseth over attempted demotion,https://www.bbc.com/news/articles/c8j31728187o...
1,2026-01-13-10-53-59 +0000,bbc,World central bank chiefs 'stand in solidarity...,https://www.bbc.com/news/articles/c041dp0z95yo...
2,2026-01-13-10-48-26 +0000,nyt,Son of Deposed Shah of Iran Seeks Center Stage...,https://www.nytimes.com/2026/01/13/world/middl...
3,2026-01-13-10-44-00 +0000,wsj,Heads of Leading Central Banks Back Fed’s Powell,https://www.wsj.com/economy/central-banking/he...
4,2026-01-13-10-38-30 +0000,nyt,Criminal Inquiry of Powell Draws Backlash From...,https://www.nytimes.com/2026/01/12/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2370/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
139,trump,59
17,iran,35
11,fed,22
13,powell,19
192,new,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
109,2026-01-13-02-08-00 +0000,wsj,Some Trump allies and officials worry that the...,https://www.wsj.com/politics/policy/trump-alli...,218
300,2026-01-12-16-36-01 +0000,nypost,GOP Sen. Thom Tillis says he will block any of...,https://nypost.com/2026/01/12/business/gop-sen...,152
222,2026-01-12-20-28-53 +0000,nyt,Deposed Shah’s Son Hopes Trump Will Put Iran R...,https://www.nytimes.com/2026/01/11/world/europ...,134
186,2026-01-12-22-04-31 +0000,cbc,Bank of Canada governor defends U.S. Fed's Pow...,https://www.cbc.ca/news/politics/bank-of-canad...,133
104,2026-01-13-02-38-00 +0000,wsj,White House Weighs Iran’s Nuclear-Talks Offer ...,https://www.wsj.com/world/middle-east/trump-ir...,118


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
109,218,2026-01-13-02-08-00 +0000,wsj,Some Trump allies and officials worry that the...,https://www.wsj.com/politics/policy/trump-alli...
276,89,2026-01-12-17-59-00 +0000,wsj,Mary Peltola will challenge GOP incumbent Sen....,https://www.wsj.com/politics/elections/mary-pe...
269,72,2026-01-12-18-16-27 +0000,nypost,Iran is ‘fully prepared for war’ if outsiders ...,https://nypost.com/2026/01/12/world-news/iran-...
287,66,2026-01-12-17-24-13 +0000,nypost,Swiss court orders bar co-owner kept in custod...,https://nypost.com/2026/01/12/world-news/swiss...
229,57,2026-01-12-20-01-04 +0000,nypost,Mark Kelly sues Pete Hegseth to stop ‘unconsti...,https://nypost.com/2026/01/12/us-news/mark-kel...
255,43,2026-01-12-18-55-41 +0000,nypost,Biden admin likely paid out $84.6M in error fo...,https://nypost.com/2026/01/12/us-news/biden-ad...
146,40,2026-01-13-00-17-31 +0000,nyt,FBI’s ICE Shooting Inquiry Examines Renee Good...,https://www.nytimes.com/2026/01/12/us/politics...
258,38,2026-01-12-18-50-00 +0000,nypost,UK probes X over Grok AI chatbot’s sexualized ...,https://nypost.com/2026/01/12/business/uk-prob...
47,37,2026-01-13-08-42-07 +0000,wsj,Stock Market Today: Dow Futures Inch Down Ahea...,https://www.wsj.com/livecoverage/stock-market-...
216,36,2026-01-12-20-46-00 +0000,wsj,President Trump is complaining to aides about ...,https://www.wsj.com/politics/policy/trump-has-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
